In [1]:
#Testing models

In [1]:
# -*- coding: UTF-8 -*-
# Local modules
import os
import sys
import argparse
# 3rd-Party Modules
import numpy as np
import pickle as pk
import pandas as pd
from tqdm import tqdm
import glob
import librosa
import copy

# PyTorch Modules
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torch.utils.data import ConcatDataset, DataLoader
import torch.optim as optim
from transformers import AutoModel
import importlib
# Self-Written Modules
sys.path.append(os.getcwd())
sys.path.append('../')
from benchmark import net
from benchmark import utils

In [9]:
os.listdir('../data_tmp/wavlm_large_meanlast4')[:3]

['MSP-PODCAST_0656_0234.pt',
 'MSP-PODCAST_2796_0625.pt',
 'MSP-PODCAST_3571_0440_0001.pt']

In [11]:
x = torch.load('../data_tmp/wavlm_large_meanlast4/MSP-PODCAST_2796_0625.pt', map_location='cpu')
y = torch.load('../data_tmp/wavlm_large/MSP-PODCAST_2796_0625.pt', map_location='cpu')

x.mean(), y.mean()

(tensor(0.1361), tensor(-0.0005))

In [2]:
# !pip install --upgrade transformers

In [3]:
import transformers
transformers.__version__

'4.47.1'

In [4]:
# ssl_model = AutoModel.from_pretrained('microsoft/wavlm-base')
# ssl_model.freeze_feature_encoder()
# ssl_model.eval();

In [5]:
# ssl_model.config.hidden_size

In [6]:
# torch.cuda.empty_cache()
# del transcriber

In [7]:
from transformers import pipeline
import numpy as np

# transcriber = pipeline("automatic-speech-recognition", model="openai/whisper-large-v3", device=0, batch_size= 64)

from transformers import AutoModelForSpeechSeq2Seq, AutoProcessor
import torch

# Load model and processor
model = AutoModelForSpeechSeq2Seq.from_pretrained("openai/whisper-large-v3").to("cuda")
processor = AutoProcessor.from_pretrained("openai/whisper-large-v3")

# Disable gradient computation
model.eval()

RuntimeError: CUDA error: out of memory
CUDA kernel errors might be asynchronously reported at some other API call,so the stacktrace below might be incorrect.
For debugging consider passing CUDA_LAUNCH_BLOCKING=1.

In [ ]:

import logging

In [ ]:
df = pd.read_csv('../benchmark/processed_labels.csv')
df.head(2)

In [ ]:
!ls ../data/Labels/labels_consensus.csv

In [ ]:
df = pd.read_csv('../data/Labels/labels_consensus.csv')
df.head(2)

In [ ]:
def transcribe(audio_files, batch_size=8):
    results = []
    for i in range(0, len(audio_files), batch_size):
        batch = audio_files[i:i + batch_size]

        # Prepare inputs
        inputs = processor(batch, sampling_rate=16000, return_tensors="pt", padding=True)

        # Move inputs to GPU
        input_features = inputs["input_features"].to("cuda")

        # Perform inference
        with torch.no_grad():
            output = model.generate(input_features)

        # Decode and store results
        transcriptions = processor.batch_decode(output, skip_special_tokens=True)
        results.extend(transcriptions)
    return results

In [ ]:
# # Prepare inputs
# inputs = processor(batch, sampling_rate=16000, return_tensors="pt", padding=True)

# # Move inputs to GPU
# input_features = inputs["input_features"].to("cuda")

# # Perform inference
# with torch.no_grad():
#     output = model.generate(input_features)

# # Decode and store results
# transcriptions = processor.batch_decode(output, skip_special_tokens=True)

In [14]:
sample = f"../data/Audios/{df.FileName.values[1]}"

y, sr = librosa.load(sample)

# result = transcriber({"sampling_rate": sr, "raw": y})["text"]  

inputs = processor([y,y], sampling_rate=16000, return_tensors="pt")#,  padding=True, max_length=3000)

# inputs["input_features"].shape

# # Move inputs to GPU
input_features = inputs["input_features"].to("cuda")

# # # Perform inference
with torch.no_grad():
    output = model.generate(input_features)

# # # Decode and store results
transcriptions = processor.batch_decode(output, skip_special_tokens=True)
transcriptions

Due to a bug fix in https://github.com/huggingface/transformers/pull/28687 transcription using a multilingual Whisper will default to language detection followed by transcription instead of translation to English.This might be a breaking change for your use case. If you want to instead always translate your audio to English, make sure to pass `language='en'`.
Passing a tuple of `past_key_values` is deprecated and will be removed in Transformers v4.43.0. You should pass an instance of `EncoderDecoderCache` instead, e.g. `past_key_values=EncoderDecoderCache.from_legacy_cache(past_key_values)`.
The attention mask is not set and cannot be inferred from input because pad token is same as eos token. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.


[" 1500 or something. It talks about the two different medical devices that they're authorizing them to use. And part one of them is for them to allow these chips to go into our bodies for medical purposes.",
 " 1500 or something. It talks about the two different medical devices that they're authorizing them to use. And part one of them is for them to allow these chips to go into our bodies for medical purposes."]

In [15]:
def transcribe_it(y):
    # y, sr = librosa.load(w_path)

    # result = transcriber({"sampling_rate": sr, "raw": y})["text"]  
    
    inputs = processor(y, sampling_rate=16000, return_tensors="pt")#,  padding=True, max_length=3000)
    
    # inputs["input_features"].shape
    
    # # Move inputs to GPU
    input_features = inputs["input_features"].to("cuda")
    
    # # # Perform inference
    with torch.inference_mode():
        output = model.generate(input_features)

    del input_features 
    
    # # # Decode and store results
    transcriptions = processor.batch_decode(output, skip_special_tokens=True)
    return transcriptions

In [16]:
 y, sr = librosa.load(sample)

transcribe_it(y)

[" 1500 or something. It talks about the two different medical devices that they're authorizing them to use. And part one of them is for them to allow these chips to go into our bodies for medical purposes."]

In [18]:
df = pd.read_csv('../test/Categorical_test.csv')
df.head(2)

,FileName,EmoClass
0,MSP-PODCAST_test3_0001.wav,C
1,MSP-PODCAST_test3_0002.wav,C


In [19]:
wav_files = [f"../data/Audios/{f}" for f in df.FileName]
file_ids = df.FileName

In [20]:
len(file_ids)

3200

In [21]:
import os

for w in wav_files:
    if(not os.path.isfile(w)):
        print('error')
        break

In [22]:
# !pip install datasets

In [23]:
# Helper function to load audio files
def load_audio(file_path):
    audio, _ = librosa.load(file_path, sr=16000)  # Ensure 16kHz sampling rate
    return audio

batch_size = 32
import tqdm
import gc
# Process in batches
results = []
c = 0
for i in tqdm.tqdm(range(0, len(wav_files), batch_size)):
    model.past_key_values = None
    batch_files = wav_files[i:i + batch_size]
    batch_ids = file_ids[i:i + batch_size]

    # Load audio files for the current batch
    audios = [load_audio(file) for file in batch_files]

    # Get transcriptions
    transcriptions = transcribe_it(audios)

    # torch.cuda.empty_cache()
    # torch.cuda.empty_cache()
    # gc.collect()
    # torch.cuda.synchronize()
    # gc.collect()

    if(c%5==0):
        torch.cuda.empty_cache()
        gc.collect()
        # torch.cuda.reset_peak_memory_stats()
        # Optional: Force CUDA device reset
        torch.cuda.reset_peak_memory_stats()
        torch.cuda.reset_accumulated_memory_stats()

    if(c%10==0):
        del model
        model = AutoModelForSpeechSeq2Seq.from_pretrained("openai/whisper-large-v3").to("cuda")
        
    print(f"GPU memory allocated: {torch.cuda.memory_allocated() / 1e9:.2f} GB // GPU memory reserved: {torch.cuda.memory_reserved()/1e9:.2f} GB")
    # del audios
    # transcriber.model.to("cpu")
    # torch.cuda.empty_cache()
    # transcriber.model.to("cuda")
    # Merge transcriptions with IDs
    for trans, file_id in zip(transcriptions, batch_ids):
        results.append({"FileName": file_id, "transcription": trans})

    c += 1

  1%|▊                                                                                  | 1/100 [00:33<54:35, 33.08s/it]

GPU memory allocated: 12.62 GB // GPU memory reserved: 12.98 GB


  2%|█▋                                                                                 | 2/100 [00:55<44:11, 27.05s/it]

GPU memory allocated: 12.62 GB // GPU memory reserved: 29.81 GB


  3%|██▍                                                                                | 3/100 [01:17<39:26, 24.39s/it]

...

GPU memory allocated: 12.62 GB // GPU memory reserved: 29.86 GB


 99%|█████████████████████████████████████████████████████████████████████████████████▏| 99/100 [37:13<00:21, 21.04s/it]

GPU memory allocated: 12.62 GB // GPU memory reserved: 29.86 GB


100%|█████████████████████████████████████████████████████████████████████████████████| 100/100 [37:35<00:00, 22.55s/it]

GPU memory allocated: 12.62 GB // GPU memory reserved: 29.90 GB


In [24]:
for i in tqdm.tqdm(range(0, len(wav_files), batch_size)):
    batch_files = wav_files[i:i + batch_size]
    batch_ids = file_ids[i:i + batch_size]


00%|██████████████████████████████████████████████████████████████████████████████| 100/100 [00:00<00:00, 12116.31it/s]

In [25]:
# del transcriber

In [26]:
# import pandas as pd

# # Convert results to DataFrame
# transcriptions_df = pd.DataFrame(results)
# transcriptions_df.head()

,FileName,transcription
0,MSP-PODCAST_test3_0001.wav,"And it's a sad thing, and I feel that's one o..."
1,MSP-PODCAST_test3_0002.wav,And this isn't getting them in for a demo. Th...
2,MSP-PODCAST_test3_0003.wav,nurse us in hospitals or in who care for us i...
3,MSP-PODCAST_test3_0004.wav,seriously i mean people don't like to be push...
4,MSP-PODCAST_test3_0005.wav,are really hardcore train writer i think they...


In [27]:
# transcriptions_df.to_csv("whisper_transcripts_test.csv", index = False)

In [29]:
transcriptions_df['l'] = transcriptions_df['transcription'].str.len()
transcriptions_df.head()

,FileName,transcription,l
0,MSP-PODCAST_test3_0001.wav,"And it's a sad thing, and I feel that's one o...",100
1,MSP-PODCAST_test3_0002.wav,And this isn't getting them in for a demo. Th...,128
2,MSP-PODCAST_test3_0003.wav,nurse us in hospitals or in who care for us i...,61
3,MSP-PODCAST_test3_0004.wav,seriously i mean people don't like to be push...,59
4,MSP-PODCAST_test3_0005.wav,are really hardcore train writer i think they...,81


In [38]:
transcriptions_df.sort_values(by='l', ascending=True).head(10)

,FileName,transcription,l
1892,MSP-PODCAST_test3_1893.wav,Ooh.,5
1229,MSP-PODCAST_test3_1230.wav,Ooh.,5
2930,MSP-PODCAST_test3_2931.wav,Yeah.,6
1428,MSP-PODCAST_test3_1429.wav,Yeah.,6
1904,MSP-PODCAST_test3_1905.wav,Yeah.,6
162,MSP-PODCAST_test3_0163.wav,Yeah.,6
2855,MSP-PODCAST_test3_2856.wav,Mm-hmm.,8
981,MSP-PODCAST_test3_0982.wav,"He says,",9
427,MSP-PODCAST_test3_0428.wav,Thank you.,11
2716,MSP-PODCAST_test3_2717.wav,gentle and,11


In [41]:
import IPython
IPython.display.Audio('../data/Audios/MSP-PODCAST_test3_1905.wav')

In [7]:
import pandas as pd
transcriptions_df = pd.read_csv("whisper_transcripts.csv")
transcriptions_df.head(),transcriptions_df.shape

(                    FileName  \
 0  MSP-PODCAST_0002_0033.wav   
 1  MSP-PODCAST_0002_0039.wav   
 2  MSP-PODCAST_0002_0051.wav   
 3  MSP-PODCAST_0002_0059.wav   
 4  MSP-PODCAST_0002_0061.wav   
 
                                        transcription  
 0               and I mean the numbers, right? So...  
 1   1500 or something. It talks about the two dif...  
 2   I did. So here it is. So it's page 1501 in th...  
 3   You know, it sounds like it could be like a b...  
 4   Yeah. So, but Molly, I mean, this is really o...  ,
 (116221, 2))

In [5]:
transcriptions_df['FileName'] = transcriptions_df['id'] 

In [8]:
# transcriptions_df[['FileName','transcription']].to_csv("whisper_transcripts.csv", index = False)

In [ ]:
!nvidia-smi

In [ ]:

torch.cuda.empty_cache()
gc.collect()
torch.cuda.reset_peak_memory_stats()
print(f"GPU memory allocated: {torch.cuda.memory_allocated() / 1e9:.2f} GB")
# Optional: Force CUDA device reset
torch.cuda.reset_peak_memory_stats()
torch.cuda.reset_accumulated_memory_stats()

In [ ]:
def force_gpu_cleanup():
    # Delete all CUDA tensors
    for obj in gc.get_objects():
        try:
            if torch.is_tensor(obj):
                if obj.is_cuda:
                    del obj
        except Exception:
            pass
    
    # Clear CUDA cache
    torch.cuda.empty_cache()
    torch.cuda.synchronize()
    
    # Force Python garbage collection
    gc.collect()
    
    # Optional: Reset CUDA device
    torch.cuda.reset_peak_memory_stats()
    if torch.cuda.is_available():
        current_device = torch.cuda.current_device()
        torch.cuda.set_device(current_device)

In [ ]:
os.system('sync') # Linux/Mac only

In [ ]:
import gc
torch.cuda.empty_cache()
gc.collect()
torch.cuda.reset_peak_memory_stats()
# Optional: Force CUDA device reset
# torch.cuda.reset_peak_memory_stats()
torch.cuda.reset_accumulated_memory_stats()
print(f"GPU memory allocated: {torch.cuda.memory_allocated() / 1e9:.2f} GB // GPU memory reserved: {torch.cuda.memory_reserved()/1e9:.2f} GB")